In [21]:
import requests


def fetch_batiment_groupe_ids(rnb_id: str) -> list:
    
    params = {
        'select': 'batiment_groupe_id',
        'limit': 5,
        'rnb_id': f"eq.{rnb_id}"
    }
    r = requests.get(f'https://api.bdnb.io/v1/bdnb/donnees/batiment_construction', params=params)

    data = r.json()
    
    return [row["batiment_groupe_id"] for row in data]


def fetch_bdnb_table(table: str, batiment_groupe_id: str):
    r = requests.get(f'https://api.bdnb.io/v1/bdnb/donnees/{table}',
                     params={'limit': 1,
                             'batiment_groupe_id': f'eq.{batiment_groupe_id}'})

    return r.json()

In [28]:
import csv
import json

new_rows = []

with open("rapprochement_results.csv", "r") as f:
    reader = csv.DictReader(f)
    histologe_list = list(reader)
    
    c = 0
    total_len = len(histologe_list)
    for row in histologe_list:
        
        c += 1
        print(f"{c}/{total_len}")
        
        new_row = {
            "ID": row['ID'],
            'rnb_ids': row['rnb_id'],
            "creation_years": [],
            "wall_materials": [],
            "clay_risks": [],
            "heaters": [],
            "dpe_class": [],
            "ges_class": []
        }
        
        
        
        for rnb_id in eval(row['rnb_id']):
            
            bg_ids = fetch_batiment_groupe_ids(rnb_id)
            bg_ids = list(set(bg_ids))
            
            if bg_ids:
                
                for bg_id in bg_ids:
                
                    # Année de construction et matériau des murs
                    data = fetch_bdnb_table('batiment_groupe_ffo_bat', bg_id)
                    for row in data:
                        new_row['creation_years'].append(row['annee_construction'])
                        new_row['wall_materials'].append(row['mat_mur_txt'])

                    # Risque de retrait/gonflement des argiles
                    data = fetch_bdnb_table('batiment_groupe_argiles', bg_id)
                    for row in data:
                        new_row['clay_risks'].append(row["alea"])

                    # Infos des diagnostics de performance énergétique
                    data = fetch_bdnb_table('rel_batiment_groupe_dpe_logement_complet', bg_id)
                    for row in data:
                        new_row['heaters'].append(row['type_generateur_chauffage'])
                        new_row['dpe_class'].append(row['classe_bilan_dpe'])
                        new_row['ges_class'].append(row["classe_emission_ges"])
                        
        new_rows.append(new_row)
        
with open("histologe_enrichie.csv", "w") as f:
    
    writer = csv.DictWriter(f, fieldnames= new_rows[0].keys())
    writer.writeheader()
    writer.writerows(new_rows)
    
        
    


                
        
        

        

        

    

1/54
2/54
3/54
4/54
5/54
6/54
7/54
8/54
9/54
10/54
11/54
12/54
13/54
14/54
15/54
16/54
17/54
18/54
19/54
20/54
21/54
22/54
23/54
24/54
25/54
26/54
27/54
28/54
29/54
30/54
31/54
32/54
33/54
34/54
35/54
36/54
37/54
38/54
39/54
40/54
41/54
42/54
43/54
44/54
45/54
46/54
47/54
48/54
49/54
50/54
51/54
52/54
53/54
54/54
